# 第7章: 例を使用する (Few-Shot Prompting)

- [レッスン](#lesson)
- [演習](#exercises)
- [例のプレイグラウンド](#example-playground)

## セットアップ

次のセットアップセルを実行して、APIキーをロードし、`get_completion`ヘルパー関数を確立します。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
from botocore.exceptions import ClientError
import json

# utilsパッケージからhintsモジュールをインポート
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r modelId
%store -r region

bedrock_client = boto3.client(service_name='bedrock-runtime', region_name=region)

In [ ]:
def get_completion(prompt, system_prompt=None, prefill=None):
    inference_config = {
        "temperature": 0.0,
        "maxTokens": 200
    }
    converse_api_params = {
        "modelId": modelId,
        "messages": [{"role": "user", "content": [{"text": prompt}]}],
        "inferenceConfig": inference_config
    }
    if system_prompt:
        converse_api_params["system"] = [{"text": system_prompt}]
    if prefill:
        converse_api_params["messages"].append({"role": "assistant", "content": [{"text": prefill}]})
    try:
        response = bedrock_client.converse(**converse_api_params)
        text_content = response['output']['message']['content'][0]['text']
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

---

## レッスン

**Claudeにどのように振る舞ってほしいか（またはどのように振る舞ってほしくないかの例を与えることは非常に効果的です**）:
- 正しい答えを得るために
- 正しい形式で答えを得るために

この種のプロンプティングは「**few shot prompting**」とも呼ばれます。また、「zero-shot」や「n-shot」、「one-shot」というフレーズに出会うこともあるでしょう。「shots」の数は、プロンプト内で使用される例の数を指します。

### 例

あなたが子供たちの質問に答える「親ボット」を作ろうとしている開発者だと想像してください。**Claudeのデフォルトの応答は非常に形式的でロボットのようです**。これは子供の心を傷つけてしまいます。

In [ ]:
# プロンプト
PROMPT = "Will Santa bring me presents on Christmas?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

あなたの望むトーンを説明する時間を取ることもできますが、理想的な応答の例をいくつか**Claudeに与える**方がはるかに簡単です。

In [ ]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: もちろん、かわいい子。歯を包んで今夜枕の下に置いておいて。朝には何かが待っているかもしれないよ。
Q: Will Santa bring me presents on Christmas?"""

# Print Claude's response
print(get_completion(PROMPT))

以下のフォーマット例では、名前と職業を抽出し、それを私たちが望む形に正確にフォーマットする方法について、Claudeにステップバイステップで指示を与えることができます。または、単に**Claudeにいくつかの正しくフォーマットされた例を提供し、Claudeがそこから外挿できるようにすることもできます**。`assistant`のターンにある`<individuals>`に注意して、Claudeを正しい方向に導きます。

In [ ]:
# プロンプトテンプレートで変数コンテンツのプレースホルダーを含む
PROMPT = """Silvermist Hollow, 魅力的な村は、特別な個人たちのグループの家でした。
その中には、地域の医療センターで外科手術の技術を革新した神経外科医、Dr. Liam Patelがいました。
Olivia Chenは、持続可能で息をのむようなデザインで村の風景を変えた革新的な建築家でした。
地元の劇場は、プロの音楽家で作曲家のEthan Kovacsによる魅惑的な交響曲で彩られていました。
地元の食材に情熱を注ぐ独学のシェフ、Isabella Torresは、ファーム・トゥ・テーブルのレストランで料理のセンセーションを生み出し、食通たちの必訪の目的地となりました。
これらの素晴らしい個人たちは、それぞれの独自の才能でSilvermist Hollowの活気ある生活のタペストリーに貢献しました。
<individuals>
1. Dr. Liam Patel [神経外科医]
2. Olivia Chen [建築家]
3. Ethan Kovacs [音楽家および作曲家]
4. Isabella Torres [シェフ]
</individuals>

町の中心には、シェフのOliver Hamiltonがファーム・トゥ・テーブルのレストラン、Green Plateで料理シーンを変革しました。Oliverの地元の有機食材を調達する献身は、食批評家や地元の人々から絶賛のレビューを得ています。
通りを下ると、ヘッドライブラリアンのElizabeth Chenがすべての人にとって歓迎される包括的な空間を作るために懸命に働いているRiverside Grove Libraryがあります。彼女の図書館の提供を拡大し、子供向けの読書プログラムを確立する努力は、町の識字率に大きな影響を与えました。
魅力的な町の広場を散策すると、美しい壁画に魅了されることでしょう。これらの傑作は、Riverside Groveの本質を捉える才能を持つ著名なアーティスト、Isabella Torresの作品です。
Riverside Groveのスポーツの成果も注目に値します。元オリンピック選手でコーチのMarcus Jenkinsのおかげです。Marcusは、自身の経験と情熱を活かして町の若者を指導し、Riverside Grove Swim Teamをいくつかの地域選手権に導きました。
<individuals>
1. Oliver Hamilton [シェフ]
2. Elizabeth Chen [司書]
3. Isabella Torres [アーティスト]
4. Marcus Jenkins [コーチ]
</individuals>

Oak Valley、魅力的な小さな町は、コミュニティに持続的な影響を与えた素晴らしい三人の個人の家です。
町の賑やかなファーマーズマーケットでは、Laura Simmons、持続可能に栽培された美味しい農産物で知られる情熱的な有機農家に出会えます。彼女の健康的な食生活を促進する献身は、町がよりエコ意識の高いライフスタイルを受け入れるきっかけとなりました。
Oak Valleyのコミュニティセンターでは、ダンスインストラクターのKevin Alvarezがすべての年齢の人々に動きの喜びをもたらしました。彼の包括的なダンスクラスは、住民の間に団結感と自己表現を育み、地元のアートシーンを豊かにしました。
最後に、Rachel O'Connor、疲れ知らずのボランティアは、さまざまな慈善活動に時間を捧げています。彼女の他者の生活を改善するための献身は、Oak Valley内に強いコミュニティ意識を生み出すのに重要な役割を果たしました。
彼女たちの独自の才能と揺るぎない献身を通じて、Laura、Kevin、RachelはOak Valleyの織物に自らを織り込み、活気に満ちた繁栄する小さな町を作る手助けをしています。"""

# Claudeの応答のプレフィル
PREFILL = "<individuals>"

# Claudeの応答を出力
print("--------------------------- フルプロンプトと変数の置換 ---------------------------")
print("ユーザーのターン:")
print(PROMPT)
print("\nアシスタントのターン:")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))

内容を変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下までスクロールして、[**Example Playground**](#example-playground)にアクセスしてください。

---

## 演習
- [演習 7.1 - 例によるメールフォーマット](#exercise-71---email-formatting-via-examples)

### 演習 7.1 - 例を用いたメールフォーマット
演習 6.2 をやり直しますが、今回は `PROMPT` を編集して、メールの「few-shot」例と適切な分類（およびフォーマット）を使用して、Claude に正しい答えを出力させます。Claude の出力の *最後* の文字がカテゴリの文字であることを望みます。

各メールの正しいカテゴリがどれか忘れた場合は、`EMAILS` リストの各メールの横にあるコメントを参照してください。

メールのカテゴリは次の通りです：
- (A) プレセールの質問
- (B) 壊れたまたは欠陥のあるアイテム
- (C) 請求に関する質問
- (D) その他（説明してください）

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) 壊れたまたは欠陥のあるアイテム
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) 購入前の質問 または (D) その他（説明してください）
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) 請求に関する質問
    "How did I get here I am not good with computer.  Halp." # (D) その他（説明してください）
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(ans, response[-1])) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

ヒントが必要な場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_7_1_hint)

まだ行き詰まっていますか？以下のセルを実行して、例の解決策を確認してください。

In [ ]:
print(hints.exercise_7_1_solution)

### おめでとうございます！

ここまでのすべての演習を解決した場合、次の章に進む準備が整いました。楽しいプロンプト作成を！

---

## 例のプレイグラウンド

これは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのように影響するかを見るためのエリアです。

In [ ]:
# プロンプト
PROMPT = "Will Santa bring me presents on Christmas?"

# Claudeの応答を出力します
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = """Please complete the conversation by writing the next line, speaking as "A".
Q: Is the tooth fairy real?
A: もちろん、かわいい子。歯を包んで今夜枕の下に置いておいて。朝には何かが待っているかもしれないよ。
Q: Will Santa bring me presents on Christmas?"""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# プロンプトテンプレートで変数コンテンツのプレースホルダーを含む
PROMPT = """Silvermist Hollow、魅力的な村は、特別な個人のグループの家でした。
その中には、地域の医療センターで外科手術の技術を革新した神経外科医、Dr. Liam Patelがいました。
Olivia Chenは、持続可能で息をのむようなデザインで村の風景を変えた革新的な建築家でした。
地元の劇場は、プロの音楽家で作曲家のEthan Kovacsによる魅惑的な交響曲で彩られていました。
地元の食材に情熱を注ぐ独学のシェフ、Isabella Torresは、ファーム・トゥ・テーブルのレストランで料理のセンセーションを生み出し、食通にとって必見の目的地となりました。
これらの素晴らしい個人は、それぞれの独自の才能を持ち、Silvermist Hollowの活気ある生活のタペストリーに貢献しました。
<individuals>
1. Dr. Liam Patel [神経外科医]
2. Olivia Chen [建築家]
3. Ethan Kovacs [音楽家および作曲家]
4. Isabella Torres [シェフ]
</individuals>

町の中心には、シェフのOliver Hamiltonがファーム・トゥ・テーブルのレストラン、Green Plateで料理シーンを変革しました。Oliverの地元の有機食材を調達する献身は、食批評家や地元の人々から絶賛のレビューを得ています。
通りを下ると、Riverside Grove図書館があり、館長のElizabeth Chenは、すべての人にとって歓迎される包括的な空間を作るために熱心に働いてきました。彼女の図書館の提供を拡大し、子供向けの読書プログラムを確立する努力は、町の識字率に大きな影響を与えました。
魅力的な町の広場を散策すると、美しい壁画に魅了されることでしょう。これらの傑作は、Riverside Groveの本質を捉える才能を持つ著名なアーティスト、Isabella Torresの作品です。
Riverside Groveのスポーツの成果も注目に値します。元オリンピック選手でコーチのMarcus Jenkinsのおかげです。Marcusは、自身の経験と情熱を活かして町の若者を指導し、Riverside Grove水泳チームをいくつかの地域選手権に導きました。
<individuals>
1. Oliver Hamilton [シェフ]
2. Elizabeth Chen [図書館員]
3. Isabella Torres [アーティスト]
4. Marcus Jenkins [コーチ]
</individuals>

Oak Valley、魅力的な小さな町は、コミュニティに持続的な影響を与えた素晴らしい三人の個人の家です。
町の賑やかなファーマーズマーケットでは、Laura Simmons、持続可能に栽培された美味しい農産物で知られる情熱的な有機農家に出会えます。彼女の健康的な食生活を促進する献身は、町がよりエコ意識の高いライフスタイルを受け入れることを促しました。
Oak Valleyのコミュニティセンターでは、熟練したダンスインストラクターのKevin Alvarezが、すべての年齢の人々に動きの喜びをもたらしました。彼の包括的なダンスクラスは、住民の間に団結感と自己表現を育み、地元のアートシーンを豊かにしました。
最後に、Rachel O'Connor、疲れを知らないボランティアは、さまざまな慈善活動に時間を捧げています。彼女の他者の生活を改善するための献身は、Oak Valley内で強いコミュニティ意識を生み出すのに重要な役割を果たしました。
彼女たちの独自の才能と揺るぎない献身を通じて、Laura、Kevin、RachelはOak Valleyの織物に自らを織り込み、活気に満ちた繁栄する小さな町を作る手助けをしています。"""

# Claudeの応答のプレフィル
PREFILL = "<individuals>"

# Claudeの応答を出力
print("--------------------------- フルプロンプトと変数の置換 ---------------------------")
print("ユーザーのターン:")
print(PROMPT)
print("\nアシスタントのターン:")
print(PREFILL)
print("\n------------------------------------- Claudeの応答 -------------------------------------")
print(get_completion(PROMPT, prefill=PREFILL))